In [2]:
import bokeh
from bokeh.io import output_notebook
from bokeh.plotting import show
import clustergram
import contextily
import geopandas
import libpysal
import matplotlib.pyplot as plt
import momepy
import osmnx
import pandas
import tqdm
import warnings
%watermark -iv
bokeh.io.output_notebook()

/tmp/ipykernel_402/342945542.py:6: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas
UsageError: Line magic function `%watermark` not found.


In [4]:
place = 'Souidania'
local_crs = 30792

In [5]:
geopandas.tools.geocode(place).explore()

## Buildings

In [6]:
buildings = osmnx.geometries.geometries_from_place(place, tags={"building": True})
buildings.head()

geometry  \
element_type osmid                                                          
way          264945759  POLYGON ((2.91174 36.70314, 2.91216 36.70325, ...   
             264945760  POLYGON ((2.91226 36.70298, 2.91217 36.70294, ...   
             264945767  POLYGON ((2.91154 36.70305, 2.91140 36.70296, ...   
             264945769  POLYGON ((2.91095 36.70134, 2.91168 36.70056, ...   
             264945770  POLYGON ((2.91182 36.70041, 2.91241 36.69982, ...   

                       amenity brand currency:DZA opening_hours operator  \
element_type osmid                                                         
way          264945759     NaN   NaN          NaN           NaN      NaN   
             264945760     NaN   NaN          NaN           NaN      NaN   
             264945767     NaN   NaN          NaN           NaN      NaN   
             264945769     NaN   NaN          NaN           NaN      NaN   
             264945770     NaN   NaN          NaN           NaN      NaN   

                       source name shop  \
element_type osmid                        
way          264945759    NaN  NaN  NaN   
             264945760    NaN  NaN  NaN   
             264945767    NaN  NaN  NaN   
             264945769    NaN  NaN  NaN   
             264945770    NaN  NaN  NaN   

                                                                    nodes  \
element_type osmid                                                          
way          264945759  [2705952108, 2705952109, 2705952110, 270595211...   
             264945760  [2705952124, 2705952125, 2705952126, 270595212...   
             264945767  [2705952191, 2705952192, 4051049146, 405104961...   
             264945769  [2705952198, 2705952199, 2705952200, 270595220...   
             264945770  [2705952206, 2705952207, 2705952208, 270595220...   

                        ... addr:street office denomination name:ar  \
element_type osmid      ...                                           
way          264945759  ...         NaN    NaN          NaN     NaN   
             264945760  ...         NaN    NaN          NaN     NaN   
             264945767  ...         NaN    NaN          NaN     NaN   
             264945769  ...         NaN    NaN          NaN     NaN   
             264945770  ...         NaN    NaN          NaN     NaN   

                       addr:housenumber healthcare content man_made ways type  
element_type osmid                                                             
way          264945759              NaN        NaN     NaN      NaN  NaN  NaN  
             264945760              NaN        NaN     NaN      NaN  NaN  NaN  
             264945767              NaN        NaN     NaN      NaN  NaN  NaN  
             264945769              NaN        NaN     NaN      NaN  NaN  NaN  
             264945770              NaN        NaN     NaN      NaN  NaN  NaN  

[5 rows x 46 columns]

In [7]:
buildings.geom_type.value_counts()

Polygon    1264
dtype: int64

In [8]:
buildings.geometry.explore(prefer_canvas = True, tiles="CartoDB Positron")

In [9]:
buildings = buildings[['geometry']].to_crs(local_crs)

In [10]:
range(len(buildings))

range(0, 1264)

In [11]:
buildings['uID'] = range(len(buildings))
buildings

geometry  \
element_type osmid                                                           
way          264945759   POLYGON ((519149.002 677713.374, 519186.605 67...   
             264945760   POLYGON ((519195.516 677695.454, 519186.861 67...   
             264945767   POLYGON ((519130.695 677703.646, 519118.001 67...   
             264945769   POLYGON ((519078.910 677513.433, 519144.096 67...   
             264945770   POLYGON ((519156.721 677410.329, 519209.481 67...   
...                                                                    ...   
             1154594824  POLYGON ((518401.593 678085.874, 518382.343 67...   
             1154594825  POLYGON ((518326.834 678112.778, 518318.810 67...   
             1154594826  POLYGON ((518383.863 678087.785, 518380.028 67...   
relation     7114627     POLYGON ((518690.457 678752.678, 518735.572 67...   
             11286910    POLYGON ((517373.396 680095.848, 517372.304 68...   

                          uID  
element_type osmid             
way          264945759      0  
             264945760      1  
             264945767      2  
             264945769      3  
             264945770      4  
...                       ...  
             1154594824  1259  
             1154594825  1260  
             1154594826  1261  
relation     7114627     1262  
             11286910    1263  

[1264 rows x 2 columns]

## Streets

In [12]:
osm_graph = osmnx.graph_from_place(place, network_type='drive')

In [13]:
osm_graph = osmnx.projection.project_graph(osm_graph, to_crs=local_crs)

In [14]:
streets = osmnx.graph_to_gdfs(osm_graph, nodes = False, node_geometry=False)

In [15]:
streets.head()

osmid lanes     ref       highway maxspeed  \
u          v          key                                                   
314761149  3799012716 0     34263614     2  CW 133     secondary       40   
           4758857554 0     34263614     2  CW 133     secondary       40   
3799012716 314761149  0     34263614     2  CW 133     secondary       40   
           392969183  0    376489960   NaN     NaN  unclassified       30   
4758857554 4758857544 0     34263614     2  CW 133     secondary       40   

                           oneway reversed   length  \
u          v          key                             
314761149  3799012716 0     False     True   23.153   
           4758857554 0     False    False  178.713   
3799012716 314761149  0     False    False   23.153   
           392969183  0     False     True  833.126   
4758857554 4758857544 0     False    False   27.982   

                                                                    geometry  \
u          v          key                                                      
314761149  3799012716 0    LINESTRING (519591.667 679896.391, 519604.626 ...   
           4758857554 0    LINESTRING (519591.667 679896.391, 519488.249 ...   
3799012716 314761149  0    LINESTRING (519604.626 679915.599, 519591.667 ...   
           392969183  0    LINESTRING (519604.626 679915.599, 519539.806 ...   
4758857554 4758857544 0    LINESTRING (519485.011 679752.776, 519468.113 ...   

                                access name junction bridge width  
u          v          key                                          
314761149  3799012716 0            NaN  NaN      NaN    NaN   NaN  
           4758857554 0            NaN  NaN      NaN    NaN   NaN  
3799012716 314761149  0            NaN  NaN      NaN    NaN   NaN  
           392969183  0    destination  NaN      NaN    NaN   NaN  
4758857554 4758857544 0            NaN  NaN      NaN    NaN   NaN

In [16]:
streets = momepy.remove_false_nodes(streets)
streets = streets[['geometry']]
streets['nID'] = range(len(streets))
streets

/srv/conda/envs/notebook/lib/python3.10/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as Nord Sahara 1959 / Sud Algerie (the single non-null crs provided).
  warnings.warn(


,geometry,nID
0,"LINESTRING (519591.667 679896.391, 519604.626 ...",0
1,"LINESTRING (519591.667 679896.391, 519488.249 ...",1
2,"LINESTRING (519604.626 679915.599, 519591.667 ...",2
3,"LINESTRING (519604.626 679915.599, 519539.806 ...",3
4,"LINESTRING (519485.011 679752.776, 519468.113 ...",4
...,...,...
1377,"LINESTRING (520170.222 677876.270, 520137.986 ...",1377
1378,"LINESTRING (520179.968 677628.516, 520187.560 ...",1378
1379,"LINESTRING (520180.922 677739.003, 520186.632 ...",1379
1380,"LINESTRING (520332.249 677887.319, 520214.059 ...",1380
